In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.cuda.is_available()

True

In [2]:
from src.vlstm_v4.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_v4/build.ninja...
Building extension module vlstm_v4...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_v4 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-c

Loading extension module vlstm_v4...


### qkvtest

In [3]:
S = 16 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float16
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
qs, qs.shape

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
           [0.3999, 0.5000, 0.6001, 0.7002],
           [0.7998, 0.8999, 1.0000, 1.0996],
           [1.2002, 1.2998, 1.4004, 1.5000],
           [1.5996, 1.7002, 1.7998, 1.9004],
           [2.0000, 2.0996, 2.1992, 2.3008],
           [2.4004, 2.5000, 2.5996, 2.6992],
           [2.8008, 2.9004, 3.0000, 3.0996],
           [3.1992, 3.3008, 3.4004, 3.5000],
           [3.5996, 3.6992, 3.8008, 3.9004],
           [4.0000, 4.1016, 4.1992, 4.3008],
           [4.3984, 4.5000, 4.6016, 4.6992],
           [4.8008, 4.8984, 5.0000, 5.1016],
           [5.1992, 5.3008, 5.3984, 5.5000],
           [5.6016, 5.6992, 5.8008, 5.8984],
           [6.0000, 6.1016, 6.1992, 6.3008]]]], device='cuda:0',
        dtype=torch.float16),
 torch.Size([1, 1, 16, 4]))

In [14]:
torch.set_printoptions(linewidth=200)
rs = qs @ ks.transpose(-1, -2)
rs, rs.shape

(tensor([[[[ 0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001],
           [ 2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992],
           [ 3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988],
           [ 5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984],
           [ 7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000],
           [ 8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016],
           [10.2031, 10.2031, 10.2031, 10.2031, 10.2031, 1

In [16]:
rs[:,:,8,8]

tensor([[13.3984]], device='cuda:0', dtype=torch.float16)

In [5]:
# pytorch version
# rs = qs @ ks.transpose(-1, -2) @ vs
# rs, rs.shape

In [6]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - float16!
B: 1, NH: 1, S: 16, DH: 4
blocksxy: 1-1, threads: 4-4
In Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
qTIdx=0|kvTIdx=0: qTile[3][3] = 1.500000
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[3][0](3) = 5.398438
qTIdx=0|kvTIdx=0: qTile[7][3] = 3.099609
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[7][0](3) = 11.804688
qTIdx=1|kvTIdx=0: qTile[3][3] = 4.699219
qTIdx=1|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=1|kvTIdx=0: sTile[3][0](3) = 18.203125
qTIdx=1|kvTIdx=0: qTile[7][3] = 6.300781
qTIdx=1|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=1|kvTIdx=0: sTile[7][0](3) = 24.593750


(tensor([[[[  9.5938,   9.5938,   9.5938,   9.5938],
           [ 35.2188,  35.2188,  35.2188,  35.2188],
           [ 60.7812,  60.7812,  60.7812,  60.7812],
           [ 86.4375,  86.4375,  86.4375,  86.4375],
           [112.0000, 112.0000, 112.0000, 112.0000],
           [137.6250, 137.6250, 137.6250, 137.6250],
           [163.1250, 163.1250, 163.1250, 163.1250],
           [188.8750, 188.8750, 188.8750, 188.8750],
           [223.8750, 223.8750, 223.8750, 223.8750],
           [275.0000, 275.0000, 275.0000, 275.0000],
           [326.5000, 326.5000, 326.5000, 326.5000],
           [377.5000, 377.5000, 377.5000, 377.5000],
           [429.0000, 429.0000, 429.0000, 429.0000],
           [480.0000, 480.0000, 480.0000, 480.0000],
           [531.0000, 531.0000, 531.0000, 531.0000],
           [582.5000, 582.5000, 582.5000, 582.5000]]]], device='cuda:0',
        dtype=torch.float16),
 torch.Size([1, 1, 16, 4]))

In [ ]:
qs

### Matmul kernel Test from vlstm_v2

In [ ]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [7]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [ ]:
S = 8 # sequence length
DH = 8 # hidden size

In [ ]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

In [ ]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

In [ ]:
matA.is_contiguous(), matB.is_contiguous()

In [ ]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

In [ ]:
matA[9]

In [ ]:
matA[9].cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

In [8]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))